# Organización de número de empleados por sector

## Limpieza de los datos

In [3]:
import os

def clean_and_save_to_new_folder(input_folder, output_folder):
    # Crear la carpeta de salida si no existe
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    # Obtener la lista de archivos CSV en la carpeta de entrada
    archivos_csv = [archivo for archivo in os.listdir(input_folder) if archivo.endswith('.csv')]

    # Datos excepcionales
    unuseful_words = ['Serie;;;2015;2016;2017;2018;2019;2020;2021;2022;2023(A)', 'empresarial']
    headers = "Serie;Codigo_municipio;Nombre;2020;2021;2022;2023\n"

    # Limpiar y guardar los archivos CSV en la carpeta de salida
    for archivo in archivos_csv:
        archivo_original = os.path.join(input_folder, archivo)
        archivo_limpiado = os.path.join(output_folder, archivo)
        with open(archivo_original, 'r') as f_input, open(archivo_limpiado, 'w') as f_output:
            f_output.write(headers) 
            delete_next = False
            for line in f_input:                    
                if not delete_next:
                    delete = False
                    for word in unuseful_words:
                        if word in line or line.strip() == ';':
                            delete = True
                    if line.strip() == '':
                        delete = False
                    elif '(A) Avance' in line:
                        line = line.replace('(A) Avance ;', '')
                    if not delete:
                        fields = line.strip().split(';')
                        f_output.write(';'.join(fields[:3] + fields[-5:]) + '\n')
                    if 'Zarzalejo' in line:
                        delete_next = True # Eliminar filas posteriores

# Rutas de las carpetas de entrada y salida
input_folder = ".\original_data"
output_folder = ".\clean_data"

# Limpiar y guardar los archivos CSV en la nueva carpeta
clean_and_save_to_new_folder(input_folder, output_folder)


## División por sectores

In [4]:
import os

def dividir_por_sector(input_folder, output_folder):
    for sector in ['primario', 'secundario', 'terciario']:
        sector_folder = os.path.join(output_folder, sector)
        if not os.path.exists(sector_folder):
            os.makedirs(sector_folder)

    # Obtener la lista de archivos CSV en la carpeta de entrada
    archivos_csv = [archivo for archivo in os.listdir(input_folder) if archivo.endswith('.csv')]

    # Mapear cada archivo a su sector correspondiente
    for archivo in archivos_csv:
        if 'agricultura' in archivo or 'ganaderia' in archivo or 'pesca' in archivo:
            sector = 'primario'
        elif 'industria' in archivo or 'construccion' in archivo or 'electricidad' in archivo or 'agua' in archivo or 'metalurgia' in archivo:
            sector = 'secundario'
        else:
            sector = 'terciario'

        # Mover el archivo a la carpeta correspondiente
        archivo_path = os.path.join(input_folder, archivo)
        nuevo_path = os.path.join(output_folder, sector, archivo)
        os.replace(archivo_path, nuevo_path)

# Rutas de las carpetas de entrada y salida
input_folder = "./clean_data"
output_folder = "./dividido_por_sector"

dividir_por_sector(input_folder, output_folder)

## Suma de los datos por sector

In [10]:
import os
import csv

# Función para leer y sumar los datos de los CSVs
def sumar_csvs(folder_path, output_path):
    # Crear un diccionario para almacenar los datos sumados
    datos_sumados = {}

    # Obtener la lista de archivos en la carpeta
    archivos = os.listdir(folder_path)

    # Iterar sobre cada archivo en la carpeta
    for archivo in archivos:
        if archivo.endswith('.csv'):
            with open(os.path.join(folder_path, archivo), 'r') as csv_file:
                csv_reader = csv.reader(csv_file, delimiter=';')
                # Saltar la primera fila que contiene los encabezados
                next(csv_reader)

                # Iterar sobre cada fila en el archivo CSV
                for row in csv_reader:
                    serie = row[0] if row else ""
                    codigo_municipio = row[1] if len(row) > 1 else ""
                    nombre = row[2] if len(row) > 2 else ""
                     # Verificar si ya existe una entrada para esta serie y código de municipio
                    key = (serie, codigo_municipio, nombre)
                    if key in datos_sumados:
                        # Sumar los valores de cada año
                        for i in range(3, len(row)):
                            datos_sumados[key][i] += int(row[i]) if row[i] else 0
                    else:
                        # Si no existe, crear una nueva entrada en el diccionario
                        datos_sumados[key] = [serie, codigo_municipio, nombre] + [int(x) if x else 0 for x in row[3:]]

    # Escribir los datos sumados en un nuevo archivo CSV
    with open(output_path, 'w', newline='') as csv_file:
        csv_writer = csv.writer(csv_file, delimiter=';')
        # Escribir los encabezados
        csv_writer.writerow(['Serie', 'Codigo_municipio', 'Nombre', '2020', '2021', '2022', '2023'])
        # Escribir los datos sumados
        for key, values in datos_sumados.items():
            csv_writer.writerow([key[0], key[1], key[2],  *values[3:7]])

# Separar por sectores
sumar_csvs('./dividido_por_sector/primario', './suma_ocupados_por_sector/suma_primario.csv')
sumar_csvs('./dividido_por_sector/secundario', './suma_ocupados_por_sector/suma_secundario.csv')
sumar_csvs('./dividido_por_sector/terciario', './suma_ocupados_por_sector/suma_terciario.csv')

## Unión de todos los datos de empleo por sector

In [13]:
import os
import csv
import re

def sumar_csvs_por_sector(folder_path, output_path):
    # Crear un diccionario para almacenar los datos sumados
    datos_sumados = {}

    # Obtener la lista de archivos en la carpeta
    archivos = sorted(os.listdir(folder_path))

    # Iterar sobre cada archivo en la carpeta
    for archivo in archivos:
        if archivo.endswith('.csv'):
            with open(os.path.join(folder_path, archivo), 'r') as csv_file:
                csv_reader = csv.reader(csv_file, delimiter=';')
                # Saltar la primera fila que contiene los encabezados
                next(csv_reader)

                # Obtener el sector del nombre del archivo
                sector = archivo.split('.')[0].split('_')[1]

                # Iterar sobre cada fila en el archivo CSV
                for row in csv_reader:
                    serie = row[0] if row else ""
                    codigo_municipio = row[1] if len(row) > 1 else ""
                    nombre = row[2] if len(row) > 2 else ""

                    # Verificar si la serie, código de municipio y nombre ya están en el diccionario
                    if (serie, codigo_municipio, nombre) not in datos_sumados:
                        datos_sumados[(serie, codigo_municipio, nombre)] = {}

                    # Obtener los datos para cada intervalo de edad
                    for i, valor in enumerate(row[3:7], start=2020):
                        clave = f"empleo_{sector}_{i}"
                        # Agregar los datos al diccionario
                        datos_sumados[(serie, codigo_municipio, nombre)][clave] = int(valor) if valor else 0

    # Escribir los datos sumados en un nuevo archivo CSV
    with open(output_path, 'w', newline='') as csv_file:
        csv_writer = csv.writer(csv_file, delimiter=';')
        # Escribir los encabezados en el archivo CSV
        claves_totales = []

        for key, values in datos_sumados.items():
            if claves_totales == []:
                claves_totales.extend(values.keys())
        claves_totales = [str(clave) for clave in claves_totales]

        # Escribir los encabezados en el archivo CSV
        csv_writer.writerow(['Serie', 'Codigo_municipio', 'Nombre', *claves_totales])

        # Escribir los datos sumados
        for key, values in datos_sumados.items():
            # Crear una lista de valores para cada clave en el orden de las claves totales
            fila = [key[0], key[1], key[2]]
            for clave in claves_totales:
                fila.append(values.get(clave, 0))
            csv_writer.writerow(fila)

# Ruta de la carpeta que contiene los archivos CSV
folder_path = './suma_ocupados_por_sector'
# Ruta del archivo CSV de salida
output_path = '../union_final/empleo_sector.csv'
# Llamar a la función para procesar los archivos CSV
sumar_csvs_por_sector(folder_path, output_path)